# Telemetry-to-Insight: Full Pipeline Demo

End-to-end: load telemetry with cuDF, retrieve filtered data, send to NIM for natural-language answers.

## Prerequisites

1. Generate synthetic data: `python data/synthetic/generate_telemetry.py --rows 100000`
2. Deploy NIM on GKE (see src/deploy/gke/) and port-forward: `kubectl port-forward service/my-nim-nim-llm 8000:8000 -n nim`

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path("..").resolve()))

from src.query.engine import TelemetryQueryEngine

data_path = Path("../data/synthetic/fleet_telemetry.parquet")
if not data_path.exists():
    raise FileNotFoundError(f"Generate data first: python data/synthetic/generate_telemetry.py")

engine = TelemetryQueryEngine(
    data_path=str(data_path),
    nim_base_url="http://localhost:8000",
    max_context_rows=500,
)
print("Engine ready. Call engine.query('your question') when NIM is running.")

In [ ]:
# Example query (uncomment when NIM is running)
# answer = engine.query("What is the max brake pressure across all vehicles?", sensor_type="can")
# print(answer)
print("Run engine.query('question') when NIM is available on localhost:8000.")